### 1. TESTE DE WEB SCRAPING

Bibliotecas usadas

In [1]:
import requests
from bs4 import BeautifulSoup
import os
from zipfile import ZipFile

Constanstes de configuração

In [2]:
URL = "https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos"
OUTPUT_PDF_PATH = "./PDFs/"
ZIP_NAME = "anexos.zip"
OUTPUT_ZIP_PATH = "./ZIPs/"

Função para pegar as URLs dos Anexos I e II

In [3]:
def get_urls_pdf(web_content):
  # Pegando o HTML do site
  soup = BeautifulSoup(web_content, 'html.parser')

  # Buscando os links dos PDFs
  urls_pdf = []
  for link in soup.find_all('a'):
    if link.string == "Anexo I." or link.string == "Anexo II.":
      urls_pdf.append(link.get('href'))
      print(f"{link.string} encontrado.")
    
  return urls_pdf

Função para fazer download dos Anexos no formato PDF

In [4]:
def download_pdf(urls):
  os.makedirs(OUTPUT_PDF_PATH, exist_ok=True)
  # Baixando os PDFs
  downloads = []
  for url in urls:
    print(f"- Acessando {url}")
    pdf_response = requests.get(url)
    
    if pdf_response.status_code == 200:
      pdf_name = os.path.basename(url)
      
      pdf_path = os.path.join(OUTPUT_PDF_PATH, pdf_name)

      print(f"Baixando: {pdf_name}")

      with open(pdf_path, "wb") as pdf_file:
        pdf_file.write(pdf_response.content)
        downloads.append(pdf_path)
    else:
      raise ValueError(f"Não foi possível acessar {url}. Status Code: {pdf_response.status_code}")
    
  return downloads

Função para compactar os anexos baixados no formato ZIP

In [5]:
def compress_pdf(pdfs):
  os.makedirs(OUTPUT_ZIP_PATH, exist_ok=True)
  # Compactando em ZIP
  with ZipFile(f'{OUTPUT_ZIP_PATH}{ZIP_NAME}', "w") as zip:
    for pdf in pdfs:
      zip.write(pdf, os.path.basename(pdf))
    print(f"Arquivos compactados em {ZIP_NAME}.")

Acessando o site: https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos e executando as funções

In [6]:
# Acessando o site
response_web = requests.get(URL)
response_web.raise_for_status()

print("Site acessado com sucesso!")

pdf_urls = get_urls_pdf(response_web.content)
  
downloads_pdf = download_pdf(pdf_urls)
  
compress_pdf(downloads_pdf)

Site acessado com sucesso!
Anexo I. encontrado.
Anexo II. encontrado.
- Acessando https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_I_Rol_2021RN_465.2021_RN624_RN625.2024.pdf
Baixando: Anexo_I_Rol_2021RN_465.2021_RN624_RN625.2024.pdf
- Acessando https://www.gov.br/ans/pt-br/acesso-a-informacao/participacao-da-sociedade/atualizacao-do-rol-de-procedimentos/Anexo_II_DUT_2021_RN_465.2021_RN624_RN625.2024.pdf
Baixando: Anexo_II_DUT_2021_RN_465.2021_RN624_RN625.2024.pdf
Arquivos compactados em anexos.zip.
